In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Importing Libraries and setting environment variables**

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!tar xf /content/drive/MyDrive/something/spark-3.4.0-bin-hadoop3.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
import sys
#os.environ["PYSPARK_PYTHON"] = "/Users/saavnbeli/opt/anaconda3/bin/python3"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars aws-java-sdk-1.11.1010.jar,aws-java-sdk-s3-1.11.1034.jar pyspark-shell'
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-class-path hadoop-aws-3.3.4.jar --jars hadoop-aws-3.3.4.jar pyspark-shell'
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.6-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, DoubleType, LongType, FloatType
from pyspark.sql.functions import col,lit
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import concat
from pyspark.sql.functions import lpad
import pyspark as pyspark
from pyspark import SparkContext

In [ ]:
import findspark
findspark.init()
findspark.find()

'/content/spark-3.4.0-bin-hadoop3'

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .getOrCreate()

## **Creating schema**

In [ ]:
dfschema = StructType([StructField('year', IntegerType(),True),
                                    StructField('month', StringType(),True),
                                    StructField('day', IntegerType(),True),
                                    StructField('weekday', StringType(),True),
                                    StructField('hour', IntegerType(),True),
                                    StructField('atm_status', StringType(),True),
                                    StructField('atm_id', StringType(),True),
                                    StructField('atm_manufacturer', StringType(),True),
                                    StructField('atm_location', StringType(),True),
                                    StructField('atm_streetname', StringType(),True),
                                    StructField('atm_street_number', IntegerType(),True),
                                    StructField('atm_zipcode', IntegerType(),True),
                                    StructField('atm_lat', FloatType(),True),
                                    StructField('atm_lon', FloatType(),True),
                                    StructField('currency', StringType(),True),
                                    StructField('card_type', StringType(),True),
                                    StructField('transaction_amount', IntegerType(),True),
                                    StructField('service', StringType(),True),
                                    StructField('message_code', StringType(),True),
                                    StructField('message_text', StringType(),True),
                                    StructField('weather_lat', FloatType(),True),
                                    StructField('weather_lon', FloatType(),True),
                                    StructField('weather_city_id', IntegerType(),True),
                                    StructField('weather_city_name', StringType(),True),
                                    StructField('temp', FloatType(),True),
                                    StructField('pressure', IntegerType(),True),
                                    StructField('humidity', IntegerType(),True),
                                    StructField('wind_speed', IntegerType(),True),
                                    StructField('wind_deg', IntegerType(),True),
                                    StructField('rain_3h', FloatType(),True),
                                    StructField('clouds_all', IntegerType(),True),
                                    StructField('weather_id', IntegerType(),True),
                                    StructField('weather_main', StringType(),True),
                                    StructField('weather_description', StringType(),True),])

## **Importing data**

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/BDT_dataset/atm_data_part1.csv", header=True, inferSchema = True, schema = dfschema)

In [ ]:
df.head()

Row(year=2017, month='January', day=1, weekday='Sunday', hour=0, atm_status='Active', atm_id='1', atm_manufacturer='NCR', atm_location='Næstved', atm_streetname='Farimagsvej', atm_street_number=8, atm_zipcode=4700, atm_lat=55.233421325683594, atm_lon=11.763270378112793, currency='DKK', card_type='MasterCard', transaction_amount=None, service=None, message_code=None, message_text='55.229919', weather_lat=11.760919570922852, weather_lon=2616038.0, weather_city_id=None, weather_city_name='281.15', temp=1014.0, pressure=87, humidity=7, wind_speed=260, wind_deg=None, rain_3h=92.0, clouds_all=500, weather_id=None, weather_main='light rain', weather_description=None)

In [ ]:
df1 = spark.read.csv("/content/drive/MyDrive/BDT_dataset/atm_data_part2.csv", header=True, inferSchema = True, schema = dfschema)

In [ ]:
df.head()

Row(year=2017, month='January', day=1, weekday='Sunday', hour=0, atm_status='Active', atm_id='1', atm_manufacturer='NCR', atm_location='Næstved', atm_streetname='Farimagsvej', atm_street_number=8, atm_zipcode=4700, atm_lat=55.233421325683594, atm_lon=11.763270378112793, currency='DKK', card_type='MasterCard', transaction_amount=None, service=None, message_code=None, message_text='55.229919', weather_lat=11.760919570922852, weather_lon=2616038.0, weather_city_id=None, weather_city_name='281.15', temp=1014.0, pressure=87, humidity=7, wind_speed=260, wind_deg=None, rain_3h=92.0, clouds_all=500, weather_id=None, weather_main='light rain', weather_description=None)

In [ ]:
spar_df = df.union(df1)

In [ ]:
spar_df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_id: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: float (nullable = true)
 |-- atm_lon: float (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: float (nullable = true)
 |-- weather_lon: float (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: string

In [ ]:
spar_df.count()

2468572

In [ ]:
spar_df.columns

['year',
 'month',
 'day',
 'weekday',
 'hour',
 'atm_status',
 'atm_id',
 'atm_manufacturer',
 'atm_location',
 'atm_streetname',
 'atm_street_number',
 'atm_zipcode',
 'atm_lat',
 'atm_lon',
 'currency',
 'card_type',
 'transaction_amount',
 'service',
 'message_code',
 'message_text',
 'weather_lat',
 'weather_lon',
 'weather_city_id',
 'weather_city_name',
 'temp',
 'pressure',
 'humidity',
 'wind_speed',
 'wind_deg',
 'rain_3h',
 'clouds_all',
 'weather_id',
 'weather_main',
 'weather_description']

### **Checking for null values**

In [ ]:
# craeting a function to check null values in dataframe
import pyspark.sql.functions as F
def count_missings(spark_df,sort=True):
    """
    Counts number of nulls and nans in each column
    """
    df = spark_df.select([F.count(F.when(F.isnan(c) | F.isnull(c), c)).alias(c) for (c,c_type) in spark_df.dtypes if c_type not in ('timestamp', 'date')]).toPandas()

    if len(df) == 0:
        print("There are no any missing values!")
        return None

    if sort:
        return df.rename(index={0: 'count'}).T.sort_values("count",ascending=False)

    return df

In [ ]:
# checking null values
count_missings(spar_df)

,count
weather_description,2468572
transaction_amount,2468572
weather_id,2468572
weather_city_id,2468572
wind_deg,2468366
message_code,2459010
service,2459009
atm_manufacturer,0
weather_lon,0
weather_main,0


## **Creating `Location` dimension**

In [ ]:
# Reading respective columns for location dimension from spar_df
dim_location = spar_df.select([spar_df.atm_location.alias("location"),
                                spar_df.atm_streetname.alias("streetname"),
                                spar_df.atm_street_number.alias("street_number"),
                                spar_df.atm_zipcode.alias("zipcode"),
                                spar_df.atm_lat.alias("lat"),
                                spar_df.atm_lon.alias("lon")]).distinct()

In [ ]:
dim_location.show(5, truncate=False)

+----------------------+----------+-------------+-------+---------+---------+
|location              |streetname|street_number|zipcode|lat      |lon      |
+----------------------+----------+-------------+-------+---------+---------+
|Daglig Brugsen Ø.Hurup|Kystvejen |51           |9560   |56.804127|10.27129 |
|Hørning               |Nørrealle |12           |8362   |56.086094|10.037351|
|Farsø                 |Torvet    |8            |9640   |56.771267|9.340144 |
|Storcenter indg. A    |Hobrovej  |452          |9200   |57.004795|9.875935 |
|Fur                   |Stenøre   |19           |7884   |56.804768|9.020021 |
+----------------------+----------+-------------+-------+---------+---------+
only showing top 5 rows



In [ ]:
dim_location.count()

109

In [ ]:
# adding column for primary key
dim_location= dim_location.withColumn("new_column",lit("ABC"))
# creating primary key
w = Window().partitionBy('new_column').orderBy(lit('A'))
dim_location= dim_location.withColumn("atm_location_id", row_number().over(w)).drop("new_column")
dim_location.printSchema()

root
 |-- location: string (nullable = true)
 |-- streetname: string (nullable = true)
 |-- street_number: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- lat: float (nullable = true)
 |-- lon: float (nullable = true)
 |-- atm_location_id: integer (nullable = false)



In [ ]:
dim_location.filter(dim_location['atm_location_id'] > 106).collect()

[Row(location='Slagelse', streetname='Mariendals Allè', street_number=29, zipcode=4200, lat=55.397518157958984, lon=11.341839790344238, atm_location_id=107),
 Row(location='Aarhus', streetname='Ceres Byen', street_number=75, zipcode=8000, lat=56.15700912475586, lon=10.193846702575684, atm_location_id=108),
 Row(location='Næstved', streetname='Farimagsgade', street_number=8, zipcode=4700, lat=55.68986511230469, lon=12.575057029724121, atm_location_id=109)]

In [ ]:
# Rearrange dim_location
location = dim_location.select("atm_location_id","location","streetname","street_number","zipcode","lat","lon")

## **Creating `Card Type`**

In [ ]:
# creating card_type dimension
dim_card_type = spar_df.select([spar_df.card_type.alias("card_type")]).distinct()

In [ ]:
 #checking count for validation hence verified
dim_card_type.count()

12

In [ ]:
# creating primary key or card_type_id column
dim_card_type= dim_card_type.withColumn("new_column",lit("ABC"))
w = Window().partitionBy('new_column').orderBy(lit('A'))
dim_card_type= dim_card_type.withColumn("card_type_id", row_number().over(w)).drop("new_column")

In [ ]:
 # Checking dataframe's top 5 rows
dim_card_type.show(5)

+------------------+------------+
|         card_type|card_type_id|
+------------------+------------+
|   Dankort - on-us|           1|
|            CIRRUS|           2|
|              VISA|           3|
|Mastercard - on-us|           4|
|           Maestro|           5|
+------------------+------------+
only showing top 5 rows



In [ ]:
card_type = dim_card_type.select("card_type_id","card_type")

## **Creating `Date` dimension**

In [ ]:
# Creating date data fame
dim_date = spar_df.select([spar_df.year.alias("year"),spar_df.month.alias("month"),spar_df.day.alias("day"),spar_df.hour.alias("hour"),spar_df.weekday.alias("weekday")]).distinct()

In [ ]:
# Checking top 20 rows
dim_date.show()

+----+--------+---+----+--------+
|year|   month|day|hour| weekday|
+----+--------+---+----+--------+
|2017| January|  1|   9|  Sunday|
|2017| January|  3|   5| Tuesday|
|2017| January|  8|  19|  Sunday|
|2017| January| 21|   3|Saturday|
|2017| January| 23|  21|  Monday|
|2017|February|  2|  19|Thursday|
|2017|February|  5|  16|  Sunday|
|2017|February| 21|  15| Tuesday|
|2017|   March|  2|   8|Thursday|
|2017|   April|  2|   2|  Sunday|
|2017|   April|  6|   8|Thursday|
|2017| January|  5|  21|Thursday|
|2017| January| 17|  17| Tuesday|
|2017| January| 22|  15|  Sunday|
|2017| January| 27|  22|  Friday|
|2017|February| 11|   7|Saturday|
|2017|February| 11|  13|Saturday|
|2017|February| 13|   3|  Monday|
|2017|February| 13|  12|  Monday|
|2017|February| 27|   5|  Monday|
+----+--------+---+----+--------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions  import date_format
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import to_date

# Creating New Month column with Integer value.
dim_date=dim_date.withColumn('month_new', date_format(to_date(col('month'),'MMMM'),'MM').cast(IntegerType()))

## adding new Month, day and hours columns with Zeroes
dim_date=dim_date.withColumn('month_new', lpad(col('month_new'),2,'0')).withColumn('day_new', lpad(col('day'),2,'0')).withColumn('hour_new', lpad(col('hour'),2,'0'))


In [ ]:
# Create a new column Full_Date_time by combining Year, new month, day, hour and "00" value to create timestamp in YYYYMMDDMI24HHMI format.
dim_date_final=dim_date.withColumn("full_date_time",concat(col('year'),col('month_new'),col('day_new'),col('hour_new'),lit('00')))

In [ ]:
# checking newly created columns
dim_date_final.show(10)

+----+--------+---+----+--------+---------+-------+--------+--------------+
|year|   month|day|hour| weekday|month_new|day_new|hour_new|full_date_time|
+----+--------+---+----+--------+---------+-------+--------+--------------+
|2017| January|  1|   9|  Sunday|       01|     01|      09|  201701010900|
|2017| January|  3|   5| Tuesday|       01|     03|      05|  201701030500|
|2017| January|  8|  19|  Sunday|       01|     08|      19|  201701081900|
|2017| January| 21|   3|Saturday|       01|     21|      03|  201701210300|
|2017| January| 23|  21|  Monday|       01|     23|      21|  201701232100|
|2017|February|  2|  19|Thursday|       02|     02|      19|  201702021900|
|2017|February|  5|  16|  Sunday|       02|     05|      16|  201702051600|
|2017|February| 21|  15| Tuesday|       02|     21|      15|  201702211500|
|2017|   March|  2|   8|Thursday|       03|     02|      08|  201703020800|
|2017|   April|  2|   2|  Sunday|       04|     02|      02|  201704020200|
+----+------

In [ ]:
# Creating primary key for the dimension with name date_id
dim_date_final= dim_date_final.withColumn("new_column",lit("ABC"))
w = Window().partitionBy('new_column').orderBy(lit('A'))
dim_date_final= dim_date_final.withColumn("date_id", row_number().over(w)).drop("new_column")

In [ ]:
# creating data from dim_date_final
date = dim_date_final.select("date_id","full_date_time","year","month","day","hour","weekday")

In [ ]:
#Finding the count for validation
date.count()

8685

In [ ]:
date.show(10)

+-------+--------------+----+--------+---+----+--------+
|date_id|full_date_time|year|   month|day|hour| weekday|
+-------+--------------+----+--------+---+----+--------+
|      1|  201701010900|2017| January|  1|   9|  Sunday|
|      2|  201701030500|2017| January|  3|   5| Tuesday|
|      3|  201701081900|2017| January|  8|  19|  Sunday|
|      4|  201701210300|2017| January| 21|   3|Saturday|
|      5|  201701232100|2017| January| 23|  21|  Monday|
|      6|  201702021900|2017|February|  2|  19|Thursday|
|      7|  201702051600|2017|February|  5|  16|  Sunday|
|      8|  201702211500|2017|February| 21|  15| Tuesday|
|      9|  201703020800|2017|   March|  2|   8|Thursday|
|     10|  201704020200|2017|   April|  2|   2|  Sunday|
+-------+--------------+----+--------+---+----+--------+
only showing top 10 rows



## **Creating `ATM` dimension**

In [ ]:
#creating new data fame with atm related columns
dim_atm = spar_df.select([spar_df.atm_id.alias("atm_number"),spar_df.atm_manufacturer.alias("atm_manufacturer"),spar_df.atm_lat.alias("lat"),spar_df.atm_lon.alias("lon")])

In [ ]:
# Checking top 3 rows
dim_atm.show(3)

+----------+----------------+--------+--------+
|atm_number|atm_manufacturer|     lat|     lon|
+----------+----------------+--------+--------+
|         1|             NCR|55.23342|11.76327|
|         2|             NCR|57.04284|9.950013|
|         2|             NCR|57.04284|9.950013|
+----------+----------------+--------+--------+
only showing top 3 rows



In [ ]:
#To add atm_location_id of dim_location df as a foreign key to the atm table, adding left join to the atm table and locatino table.
dim_atm = dim_atm.join(location, on = ["lat","lon"],how = "leftouter")

In [ ]:
# Taking distinct values to avoid repeated values
atm_distinct =dim_atm.distinct()

In [ ]:
#adding our primary key to the 156 sets of data
atm_distinct= atm_distinct.withColumn("new_column",lit("ABC"))
w = Window().partitionBy('new_column').orderBy(lit('A'))
atm_distinct= atm_distinct.withColumn("atm_id", row_number().over(w)).drop("new_column")

In [ ]:
# creating atm from atm_distinct
atm = atm_distinct.select('atm_id','atm_number','atm_manufacturer','atm_location_id')

In [ ]:
#Rechecking the count for validation
atm.count()

154

## **Creating `fact table`**

In [ ]:
# Creating alias
spar_df = spar_df.alias('spar_df')
date = date.alias('date')
dim_card_type = dim_card_type.alias('dim_card_type')
dim_location = dim_location.alias('dim_location')
atm = atm.alias('atm')

##### Note: Creating fact table will take 4 steps by outer left joining the input table with dimension tables
##### Note: Dropping columns as required except primary keys of dimension table as they will act as foreign key

##### Creating first_df

In [ ]:
# Creating firts_df by left join of date dimension on input data frame and dropping columns
first_df =spar_df.join(date, on = ['year','month','day','hour','weekday'],how='left').select('spar_df.*','date.date_id').drop(*['year','month','day','hour','weekday'])

In [ ]:
#  Creating alias for first
first_df = first_df.alias("first_df")

In [ ]:
# Checking count for first step for validation
first_df.count()

2468572

##### Creating second_df

In [ ]:
# Creating second_df by joining card_type dimension with first_df
second_df = first_df.join(dim_card_type, on = ['card_type'], how = 'left').select('first_df.*','dim_card_type.card_type_id').drop(*['card_type'])

In [ ]:
# Checking schema
second_df.printSchema()

root
 |-- atm_status: string (nullable = true)
 |-- atm_id: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: float (nullable = true)
 |-- atm_lon: float (nullable = true)
 |-- currency: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: float (nullable = true)
 |-- weather_lon: float (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: string (nullable = true)
 |-- temp: float (nullable = true)
 |-- pressure: integer (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- wind_speed: integer (nullable = true)
 |-- wind_deg: integer (nullable = true)
 |-- ra

In [ ]:
#  Creating alias
second_df = second_df.alias('second_df')

##### Creating third_df

In [ ]:
# Creating third_df by joining location dimension with second_df by performing outer join
third_df = second_df.withColumnRenamed('atm_location','location').withColumnRenamed('atm_lat','lat').withColumnRenamed('atm_lon','lon').withColumnRenamed('atm_streetname','streetname').withColumnRenamed('atm_street_number','street_number').withColumnRenamed('atm_zipcode','zipcode').join(dim_location, on = ['location','lat','lon','streetname','street_number','zipcode'],how = 'left').select('second_df.*','dim_location.atm_location_id').drop(*['location','lat','lon','streetname','street_number','zipcode'])

In [ ]:
# checking count again
third_df.count()

2468572

In [ ]:
# Creating alias
third_df= third_df.alias('third_df')

In [ ]:
# Checking schema
third_df.printSchema()

root
 |-- atm_status: string (nullable = true)
 |-- atm_id: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: float (nullable = true)
 |-- weather_lon: float (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: string (nullable = true)
 |-- temp: float (nullable = true)
 |-- pressure: integer (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- wind_speed: integer (nullable = true)
 |-- wind_deg: integer (nullable = true)
 |-- rain_3h: float (nullable = true)
 |-- clouds_all: integer (nullable = true)
 |-- weather_id: integer (nullable = true)
 |-- weather_main: string (nullable = true)
 |-- weather_description: string (nullable = true)
 |-- date_id: integer (nullable = true)
 |-- car

In [ ]:
# Renaming atm_id as atm_number as atm_id imported from input df
third_df = third_df.withColumnRenamed('atm_id',"atm_number")

In [ ]:
# Checking schema
third_df.printSchema()

root
 |-- atm_status: string (nullable = true)
 |-- atm_number: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: float (nullable = true)
 |-- weather_lon: float (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: string (nullable = true)
 |-- temp: float (nullable = true)
 |-- pressure: integer (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- wind_speed: integer (nullable = true)
 |-- wind_deg: integer (nullable = true)
 |-- rain_3h: float (nullable = true)
 |-- clouds_all: integer (nullable = true)
 |-- weather_id: integer (nullable = true)
 |-- weather_main: string (nullable = true)
 |-- weather_description: string (nullable = true)
 |-- date_id: integer (nullable = true)
 |--

##### Creating fourth_df

In [ ]:
# Craeting fourth_df by left joining of third with atm dimension
fourth_df= third_df.join(atm,on =['atm_number','atm_manufacturer','atm_location_id'],how ='left').select('third_df.*','atm.atm_id').drop(*['atm_manufacturer','atm_nummber'])

In [ ]:
# Checking count for validation hence verifed
fourth_df.count()

2468572

In [ ]:
# Checking schema
fourth_df.printSchema()

root
 |-- atm_location_id: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: float (nullable = true)
 |-- weather_lon: float (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: string (nullable = true)
 |-- temp: float (nullable = true)
 |-- pressure: integer (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- wind_speed: integer (nullable = true)
 |-- wind_deg: integer (nullable = true)
 |-- rain_3h: float (nullable = true)
 |-- clouds_all: integer (nullable = true)
 |-- weather_id: integer (nullable = true)
 |-- weather_main: string (nullable = true)
 |-- weather_description: string (nullable = true)
 |-- date_id: integer (nullable = true)
 |-- card_type_id: integer (nullable = true)
 

In [ ]:
# top 3 rows
fourth_df.show(3)

+---------------+----------+--------+------------------+-------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+--------------------+-------------------+-------+------------+------+
|atm_location_id|atm_status|currency|transaction_amount|service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|        weather_main|weather_description|date_id|card_type_id|atm_id|
+---------------+----------+--------+------------------+-------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+--------------------+-------------------+-------+------------+------+
|             95|    Active|     DKK|              null|   null|        null|   55.229919|   11.76092|  2616038.0|       

In [ ]:
# Rechecking count
fourth_df.count()

2468572

##### Final step creating fact table fact_atm_trans

In [ ]:
# creating fact table from fourth_df
fact_atm_trans = fourth_df.alias("fact_atm_trans")

In [ ]:
#adding our primary key to fact table
fact_atm_trans= fact_atm_trans.withColumn("new_column",lit("ABC"))
w = Window().partitionBy('new_column').orderBy(lit('A'))
fact_atm_trans= fact_atm_trans.withColumn("trans_id", row_number().over(w)).drop("new_column")

In [ ]:
# checking schema of fact table
fact_atm_trans.printSchema()

root
 |-- atm_location_id: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: float (nullable = true)
 |-- weather_lon: float (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: string (nullable = true)
 |-- temp: float (nullable = true)
 |-- pressure: integer (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- wind_speed: integer (nullable = true)
 |-- wind_deg: integer (nullable = true)
 |-- rain_3h: float (nullable = true)
 |-- clouds_all: integer (nullable = true)
 |-- weather_id: integer (nullable = true)
 |-- weather_main: string (nullable = true)
 |-- weather_description: string (nullable = true)
 |-- date_id: integer (nullable = true)
 |-- card_type_id: integer (nullable = true)
 

In [ ]:
 # dropping irrelevant columns as per schema
fact_atm_trans = fact_atm_trans.drop('weather_lat','weather_lon','weather_city_id','weather_city_name','temp','pressure','humidity','wind_speed','wind_deg')

In [ ]:
# Renaming atm_location_id as weather_loaction_id as per schema
fact_atm_trans = fact_atm_trans.withColumnRenamed("atm_location_id","weather_loc_id")

In [ ]:
# Checking final schema
fact_atm_trans.printSchema()

root
 |-- weather_loc_id: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- rain_3h: float (nullable = true)
 |-- clouds_all: integer (nullable = true)
 |-- weather_id: integer (nullable = true)
 |-- weather_main: string (nullable = true)
 |-- weather_description: string (nullable = true)
 |-- date_id: integer (nullable = true)
 |-- card_type_id: integer (nullable = true)
 |-- atm_id: integer (nullable = true)
 |-- trans_id: integer (nullable = false)



In [ ]:
fact_atm_trans1 = fact_atm_trans.select('trans_id','atm_id','weather_loc_id','date_id','card_type_id','atm_status','currency','service','transaction_amount','message_code','message_text','rain_3h','clouds_all','weather_id','weather_main','weather_description')

In [ ]:
fact_atm_trans1 = fact_atm_trans1.alias('fact_atm_trans1')


In [ ]:
fact_atm_trans1.show(2)

+--------+------+--------------+-------+------------+----------+--------+-------+------------------+------------+------------+-------+----------+----------+-------------+-------------------+
|trans_id|atm_id|weather_loc_id|date_id|card_type_id|atm_status|currency|service|transaction_amount|message_code|message_text|rain_3h|clouds_all|weather_id| weather_main|weather_description|
+--------+------+--------------+-------+------------+----------+--------+-------+------------------+------------+------------+-------+----------+----------+-------------+-------------------+
|       1|    84|            33|    694|           4|    Active|     DKK|   null|              null|        null|      57.048|    0.0|       800|      null| Sky is Clear|               null|
|       2|    20|            83|   4617|           7|    Active|     DKK|   null|              null|        null|   56.972301|   92.0|       501|      null|moderate rain|               null|
+--------+------+--------------+-------+-----

In [ ]:
fact_atm_trans1.printSchema()

root
 |-- trans_id: integer (nullable = false)
 |-- atm_id: integer (nullable = true)
 |-- weather_loc_id: integer (nullable = true)
 |-- date_id: integer (nullable = true)
 |-- card_type_id: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- service: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- rain_3h: float (nullable = true)
 |-- clouds_all: integer (nullable = true)
 |-- weather_id: integer (nullable = true)
 |-- weather_main: string (nullable = true)
 |-- weather_description: string (nullable = true)



## **Writing csv**

##### **Saving fact table as csv**

In [ ]:
fact_atm_trans1.coalesce(1).write.mode("overwrite").option("header", "true").csv("/content/drive/MyDrive/BDT_dataset/Cleaned dataset/fact.csv")

##### **Dimension tables**

In [ ]:
#dim-atm/
atm.coalesce(1).write.mode("overwrite").option("header", "true").csv("/content/drive/MyDrive/BDT_dataset/Cleaned dataset/atm.csv")

In [ ]:
### dim-card-type/
card_type.coalesce(1).write.mode("overwrite").option("header", "true").csv("/content/drive/MyDrive/BDT_dataset/Cleaned dataset/card_type.csv")

In [ ]:
#dim-date
date.coalesce(1).write.mode("overwrite").option("header", "true").csv("/content/drive/MyDrive/BDT_dataset/Cleaned dataset/data.csv")

In [ ]:
#dim-location
location.coalesce(1).write.mode("overwrite").option("header", "true").csv("/content/drive/MyDrive/BDT_dataset/Cleaned dataset/location.csv")